In [ ]:
#Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import random
import requests
import json

#Installing citipy
from citipy import citipy

#API key
from api_key import api_key
#print(api_key)

In [ ]:
#Seeding random values, if needed
random.seed(100)

#Empty lists for iteration
locat = [] #to store names of cities
t_lat = [] #to store dummy latitude values, in case needed
t_lng = [] #to store dummy longitude values, in case needed

#Generate random numbers for coordinates, then pass through citipy
#Initial range must be larger than target value, as duplicate cities will populate results
for i in range(1600):    
    #Generate random values according to maximum values for lat and lng
    rn_lat = round(random.uniform(-90, 90), 4)
    rn_lng = round(random.uniform(-180, 180), 4)
    
    #Using citipy to find nearest city to generated coordinates
    coord = citipy.nearest_city(rn_lat, rn_lng)

    #Finding name of city and its country
    name_cit = coord.city_name
    name_con = coord.country_code
    
    #Append values to lists
    locat.append(f'{name_cit},{name_con}')
    t_lat.append(rn_lat)
    t_lng.append(rn_lng)
    
#print(locat)

In [ ]:
#Removing duplicate values
locat = list(set(locat))

len(locat)
#print(locat)

In [ ]:
#Generate geolocation query links to pass through API

#Base url for geolocation
url_geo = 'http://api.openweathermap.org/geo/1.0/direct?q='

#Empty lists for iteration
q_geo = [] #to store geolocation query urls
t_city = [] #to store city name, not final
t_country = [] #to store country, not final

#Generating query urls for geolocation
for l in locat:
    #Generating urls with each city country pair
    q_geo.append(url_geo + l + '&appid=' + api_key)
    
    #Splitting city and country and appending to lists
    locat_split = l.split(',')
    t_city.append(locat_split[0])
    t_country.append(locat_split[1])
    
print(len(q_geo))
print(len(t_city))
print(len(t_country))
#q_geo

In [ ]:
#Generating API responses (takes around 4 minutes+)

gg = [] #to store json responses, needed to confirm success of API calls

#Generating API calls
for qg in q_geo:
    georesponse = requests.get(qg)
    geodata = georesponse.json()
    gg.append(geodata)
             
print(len(gg))

In [ ]:
gg[0]

In [ ]:
#Accounting for blank entries

#Empty lists for iteration
lat = [] #to store latitude
lng = [] #to store longitude
city = [] #to store cities from JSON response
country = [] #to store countries from JSON response

for g in gg:
    #If response is empty, remove from response list
    if g == []:
        blankindex = gg.index(g)
        del gg[blankindex]
    
    #Append actual city data to empty lists
    else:
        true_lat = g[0]['lat']
        true_lng = g[0]['lon']
        true_cit = g[0]['name']
        true_con = g[0]['country']
        
        lat.append(true_lat)
        lng.append(true_lng)
        city.append(true_cit)
        country.append(true_con)

print(len(city))
print(len(country))
print(len(lat))

In [ ]:
#DataFrame for city information
cities_df = pd.DataFrame({'City': city,
                         'Country': country,
                         'Lat': lat,
                         'Lng': lng})

#cities_df.head()

In [ ]:
#Generate weather data query links to pass through API

#Base url for weather data
url_wthr = 'https://api.openweathermap.org/data/2.5/weather?'

#Empty list for iteration
q_wthr = [] #to store weather query urls

#Generate query urls for weather data
for w in range(len(lat)):
    
    #Generate urls with actual lat and lng values, and using imperial units
    q_lat = 'lat=' + str(lat[w])
    q_lng = '&lon=' + str(lng[w])
    unit = '&units=imperial'
    q_wthr.append(url_wthr + q_lat + q_lng + unit+ '&appid=' + api_key)
    
#len(q_wthr)

In [ ]:
#Finding temperature, humidity, cloudiness, and wind speed of all cities (takes around 5 minutes+)

#Empty lists for iteration
temperatures = []
#params = main, temp
humidity = []
#params = main, humidity
cloudiness = []
#params = clouds, all
windspeed = []
#params = wind, speed

#Generate API calls
for qw in q_wthr:
    wthresponse = requests.get(qw)
    wthdata = wthresponse.json()
    
    tempf = wthdata['main']['temp']
    humid = wthdata['main']['humidity']
    cloud = wthdata['clouds']['all']
    winds = wthdata['wind']['speed']
    
    temperatures.append(tempf)
    humidity.append(humid)
    cloudiness.append(cloud)
    windspeed.append(winds)

#print(temperatures)
#print(humidity)
#print(cloudiness)
#print(windspeed)

In [ ]:
#Attach new data to dataframe
cities_df['Current Temp (F)'] = temperatures
cities_df['Humidity (%)'] = humidity
cities_df['Cloudiness (%)'] = cloudiness
cities_df['Wind Speed (mph)'] = windspeed

#cities_df.head()

In [ ]:
## Plotting

In [ ]:
#Scatter plot displaying temperature across latitude
#Using absolute values of latitude, not comparing north vs south
plt.scatter(abs(cities_df['Lat']), cities_df['Current Temp (F)'])

plt.xlim(0, 90)
plt.ylim(0, 100)

plt.xlabel('Latitude')
plt.ylabel('Degrees Fahrenheit')

plt.title('Temperature (F) across Latitude')
plt.grid()
plt.show()

#The plot compares temperature readings of cities with their latitude, and shows a strong correlation.  
#The closer one gets to 0 degrees lat (the equator), the higher the temperature will be.

In [ ]:
#Scatter plot displaying humidity across latitude
#Using absolute values of latitude, not comparing north vs south
plt.scatter(abs(cities_df['Lat']), cities_df['Humidity (%)'])

plt.xlim(0, 90)
plt.ylim(0, 100)

plt.xlabel('Latitude')
plt.ylabel('Relative Humidity')

plt.title('Humidity across Latitude')
plt.grid()
plt.show()

#The plot compares humidity readings of cities with their latitude, and shows a weak correlation.
#Generally, humidity seems to be high across most cities regardless of latitude.

In [ ]:
#Scatter plot displaying cloud cover across latitude
#Using absolute values of latitude, not comparing north vs south
plt.scatter(abs(cities_df['Lat']), cities_df['Cloudiness (%)'])

plt.xlim(0, 90)
plt.ylim(0, 110)

plt.xlabel('Latitude')
plt.ylabel('Cloud Cover')

plt.title('Cloudiness across Latitude')
plt.grid()
plt.show()

#The plot compares the cloud cover of cities with their latitude, and shows a weak correlation.
#Cloud cover doesn't appear to be dependent on the latitude of a location.

In [ ]:
#Scatter plot displaying wind speed across latitude
#Using absolute values of latitude, not comparing north vs south
plt.scatter(abs(cities_df['Lat']), cities_df['Wind Speed (mph)'])

plt.xlim(0, 90)
plt.ylim(0, 50)

plt.xlabel('Latitude')
plt.ylabel('Cloud Cover')

plt.title('Wind Speed across Latitude')
plt.grid()
plt.show()

#The plot compares wind speed readings of cities with their latitude, and shows a weak correlation.
#Generally, there is more potential for higher wind speeds between latitudes 20 and 60.

In [ ]:
## Plotting, Comparing Northern and Southern Hemispheres

In [ ]:
#Separating dataframe by latitude into two
no_hem = cities_df.loc[cities_df['Lat'] >= 0]
so_hem = cities_df.loc[cities_df['Lat'] <= 0]

#no_hem.shape
#so_hem.shape

In [ ]:
#Scatter plots comparing temperatures in northern and southern hemispheres
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.suptitle('Temperature (F) across Latitude')

ax1.scatter(no_hem['Lat'], no_hem['Current Temp (F)'])
ax1.set(xlabel='Latitude', xlim=(0, 90), ylabel='Degrees Fahrenheit', ylim=(0, 100), title='Northern Hemisphere')
ax1.grid()

ax2.scatter(so_hem['Lat'], so_hem['Current Temp (F)'])
ax2.set(xlabel='Latitude', xlim=(-90, 0), ylabel='Degrees Fahrenheit', ylim=(0, 100), title='Southern Hemisphere')
ax2.grid()

plt.subplots_adjust(right=1.5, top=0.8, wspace=0.5)
plt.show()

In [ ]:
#Scatter plots comparing humidity in northern and southern hemispheres
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.suptitle('Humidity across Latitude')

ax1.scatter(no_hem['Lat'], no_hem['Humidity (%)'])
ax1.set(xlabel='Latitude', xlim=(0, 90), ylabel='Relative Humidity', ylim=(0, 100), title='Northern Hemisphere')
ax1.grid()

ax2.scatter(so_hem['Lat'], so_hem['Humidity (%)'])
ax2.set(xlabel='Latitude', xlim=(-90, 0), ylabel='Relative Humidity', ylim=(0, 100), title='Southern Hemisphere')
ax2.grid()

plt.subplots_adjust(right=1.5, top=0.8, wspace=0.5)
plt.show()

In [ ]:
#Scatter plots comparing cloudiness in northern and southern hemispheres
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.suptitle('Cloudiness across Latitude')

ax1.scatter(no_hem['Lat'], no_hem['Cloudiness (%)'])
ax1.set(xlabel='Latitude', xlim=(0, 90), ylabel='Cloudiness', ylim=(0, 100), title='Northern Hemisphere')
ax1.grid()

ax2.scatter(so_hem['Lat'], so_hem['Cloudiness (%)'])
ax2.set(xlabel='Latitude', xlim=(-90, 0), ylabel='Cloudiness', ylim=(0, 100), title='Southern Hemisphere')
ax2.grid()

plt.subplots_adjust(right=1.5, top=0.8, wspace=0.5)
plt.show()

In [ ]:
#Scatter plots comparing wind speed in northern and southern hemispheres
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.suptitle('Wind Speed across Latitude')

ax1.scatter(no_hem['Lat'], no_hem['Wind Speed (mph)'])
ax1.set(xlabel='Latitude', xlim=(0, 90), ylabel='Wind Speed (mph)', ylim=(0, 50), title='Northern Hemisphere')
ax1.grid()

ax2.scatter(so_hem['Lat'], so_hem['Wind Speed (mph)'])
ax2.set(xlabel='Latitude', xlim=(-90, 0), ylabel='Wind Speed (mph)', ylim=(0, 50), title='Southern Hemisphere')
ax2.grid()

plt.subplots_adjust(right=1.5, top=0.8, wspace=0.5)
plt.show()